In [1]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so
from visualization_helper import *

In [2]:
import numpy as np

### Initial Deployment

In [3]:
name = 'hemisphere'

In [4]:
input_path = '../../data/{}.json'.format(name)
target_mesh_path = '../../data/target_meshes/{}.obj'.format(name)

In [5]:
# input_path = '../../data/{}.json'.format('hemisphere')
# target_mesh_path = '../../data/target_meshes/{}.obj'.format('hemisphere')

In [6]:
from helpers.deployment_helper import get_deployed_umbrella_mesh

In [7]:
from load_jsondata import read_data
input_data, io = read_data(filepath = input_path)
width = 2*input_data['arm_plate_edge_offset']
thickness = width * 0.5 # 1.5 mm # FIX from mm to meters everywhere
cross_section = [thickness, width]

In [8]:
curr_um = umbrella_mesh.UmbrellaMesh(target_mesh_path, io)
curr_um.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))

In [9]:
curr_um.energy(UmbrellaEnergyType.Full)

7.589897059399548e-21

#### Pin Rigid Motion



In [10]:
use_pin = False

In [11]:
driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [12]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 300
OPTS.verboseNonPosDef = False

In [13]:
rod_colors = []
for ri in range(curr_um.numSegments()):
    rod_colors.append(np.ones(curr_um.segment(ri).rod.numVertices()) * ri)

In [14]:
import mesh
target_mesh = mesh.Mesh(target_mesh_path)
view = linkage_vis.LinkageViewerWithSurface(curr_um, target_mesh, width=1024, height=600)
# view = linkage_vis.LinkageViewer(curr_um, width=1024, height=600)
# view.update(scalarField = rod_colors)
set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
view.averagedMaterialFrames = True
view.showScalarField(rod_colors)
view.show()

Renderer(camera=PerspectiveCamera(aspect=1.7066666666666668, children=(PointLight(color='#999999', position=(0…

In [15]:
angles = []
def eqm_callback(prob, i):
    angles.append(curr_um.getDoFs()[curr_um.jointAngleDoFIndices()])
    if (i % 10 == 0):
        view.update()
        view.showScalarField(rod_colors)

In [49]:
curr_um.uniformDeploymentEnergyWeight = 0.1
# curr_um.deploymentForceType = umbrella_mesh.DeploymentForceType.Constant
curr_um.targetDeploymentHeight = thickness * 50
curr_um.repulsionEnergyWeight = 0
curr_um.attractionWeight = 0
curr_um.setHoldClosestPointsFixed(False)
curr_um.scaleInputPosWeights(0.5)

In [50]:
curr_um.energyElastic(), curr_um.energyDeployment(), curr_um.energyRepulsion(), curr_um.energyAttraction()

(1864.5883320645164, 34229.59369078609, 0.0, 0.0)

In [51]:
dof = curr_um.getDoFs()
for i in range(curr_um.numJoints()):
    if (curr_um.joint(i).jointType() == umbrella_mesh.JointType.X):
        dof[curr_um.dofOffsetForJoint(i) + 6] = 1e-3
curr_um.setDoFs(dof)

In [52]:
angles = []
def eqm_callback(prob, i):
    angles.append(curr_um.getDoFs()[curr_um.jointAngleDoFIndices()])
    if (i % 10 == 0):
        view.update()
        view.showScalarField(rod_colors)

In [55]:
results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

0	24511.6	1412.14	1412.14	0	1
Added constraint 1128 to working set
Added constraint 1131 to working set
Added constraint 1132 to working set
Added constraint 1164 to working set
Added constraint 1166 to working set
Added constraint 1167 to working set
Added constraint 1169 to working set
Added constraint 1177 to working set
Added constraint 1181 to working set
Added constraint 1182 to working set
Added constraint 1230 to working set
Added constraint 1241 to working set
Added constraint 1252 to working set
1	24511.6	1412.14	916.16	0	1
Added constraint 1246 to working set
2	24511.6	1412.14	871.994	5.66297e-16	1
Added constraint 1184 to working set
3	24511.6	1412.14	862.273	4.77823e-14	1
Added constraint 1133 to working set
4	24511.6	1412.14	785.1	3.76595e-14	1
Added constraint 1251 to working set
5	24511.6	1412.14	696.517	1.05234e-13	1
Added constraint 1245 to working set
6	24511.6	1412.14	669.409	4.1714e-14	1
Added constraint 1226 to working set
7	24511.6	1412.14	583.667	5.10387e-15	1
A

Bound constraint on variable 12838 reencounteredstep component: 3.2108e-10
g_free component: 0
throwing logic error (this freezes Knitro!!!)


RuntimeError: Re-encountered bound in working set

In [21]:
results.success

True

### Initialize Design Optimization

In [22]:
import umbrella_optimization
import umbrella_optimization_finite_diff

In [23]:
optimizer = umbrella_optimization.UmbrellaOptimization(curr_um, OPTS, 2.5, -1, False, fixedVars)

In [24]:
optimizer.beta = 5 * 10e5

In [25]:
optimizer.gamma

1.0

In [26]:
optimizer.set_holdClosestPointsFixed(False)

In [27]:
optimizer.J(curr_um.getDesignParameters(), umbrella_optimization.OptEnergyType.Umbrella)

0.9999999999999906

In [28]:
optimizer.J(curr_um.getDesignParameters(), umbrella_optimization.OptEnergyType.Target)

53.67434835911158

In [29]:
optimizer.J(curr_um.getDesignParameters(), umbrella_optimization.OptEnergyType.Full)

54.674348359111576

In [30]:
original_design_parameters = curr_um.getDesignParameters()

In [31]:
rest_height_optimizer = umbrella_optimization.UmbrellaRestHeightsOptimization(optimizer)

In [32]:
rest_height_optimizer.gradp_J()

array([-0.00632466,  1.65755653, -2.23491565,  2.26207482,  0.05154824,
       -1.70959191, -3.81370241, -0.61622033, -0.24436055,  0.45930648,
       -3.26697478, -1.68361762,  0.62587767, -4.14823099,  0.61390905,
       -0.18648684,  0.57538401, -0.13090394, -0.92004069, -2.79706783,
       -0.51380654,  1.57813311, -1.37434514,  0.28913641, -0.82584089,
        0.00510864,  0.33042519, -0.29870306, -3.66890537,  2.30574845,
        1.87331508,  0.3286631 , -3.45546092, -1.72898905, -3.51414843,
       -0.48998991,  2.06565789,  0.72738685, -1.39738509,  2.08220953,
       -1.84511146,  0.65615038,  2.10981206,  4.23983063, -0.39149454,
       -1.39899525, -1.38028077,  3.96411728,  2.26348559, -0.17117002,
        4.58586004, -1.34534165, -0.27863493, -0.20392469, -0.38280249,
       -0.15979987,  3.44904131,  1.35297002, -1.27570985,  1.3288103 ,
       -1.49925887, -0.00631462, -0.48108868, -1.24895256, -1.48199838,
        3.42689603, -1.81196532, -1.7122842 ,  0.1658546 ,  3.86

In [33]:
class UmbrellaOptimizationCallback:
    def __init__(self, optimizer, umbrella, view, update_color = False, no_surface = False, callback_freq = 1):
        self.optimizer     = optimizer
        self.umbrella       = umbrella
        self.view  = view
        self.update_color = update_color
        self.no_surface    = no_surface
        self.callback_freq = callback_freq
        self.iterateData = []

    def __call__(self):
        if self.view and (len(self.iterateData) % self.callback_freq == 0):
            if self.update_color:
                self.view.showScalarField(rod_colors)
            else:
                pass
                self.view.update()
        return

    def numIterations(self): return len(self.iterateData)

In [34]:
algorithm = umbrella_optimization.OptAlgorithm.NEWTON_CG
cb = UmbrellaOptimizationCallback(optimizer, curr_um, view, True, False, 1)

In [35]:
optimizer.getAttractionWeight(), optimizer.beta

(20000.0, 5000000.0)

In [36]:
solverStatus = umbrella_optimization.optimize(rest_height_optimizer, algorithm, 20, 1.0, 1e-2, cb, -1)

In [37]:
solverStatus

1

In [38]:
curr_um.getDesignParameters()

array([40.62096911, 50.78566104, 40.62096911, 38.92980297, 38.24880759,
       40.77835526, 38.24880759, 43.12143542, 46.32201977, 38.92980297,
       46.32201977, 58.8264265 , 46.32201977, 37.34163296, 36.93303243,
       43.12143542, 36.93303243, 37.44374091, 37.29110627, 37.34163296,
       37.29110627, 46.7319472 , 48.62571257, 52.99146035, 48.62571257,
       50.78566104, 52.84103319, 47.91184761, 52.84103319, 40.77835526,
       52.84103319, 64.14603784, 63.90425505, 50.78566104, 63.90425505,
       71.31416061, 63.90425505, 58.8264265 , 59.50627071, 64.14603784,
       59.50627071, 43.12143542, 59.50627071, 60.99872726, 59.30433709,
       58.8264265 , 59.30433709, 67.49970524, 59.30433709, 46.7319472 ,
       48.79928287, 60.99872726, 48.79928287, 37.44374091, 48.79928287,
       42.32869596, 42.75745781, 46.7319472 , 42.75745781, 45.51629814,
       42.87837875, 41.87023972, 42.87837875, 52.99146035, 55.15927549,
       43.30802753, 55.15927549, 47.91184761, 55.15927549, 66.50

In [39]:
rest_height_optimizer.params()

array([40.62096911, 38.24880759, 46.32201977, 36.93303243, 37.29110627,
       48.62571257, 52.84103319, 63.90425505, 59.50627071, 59.30433709,
       48.79928287, 42.75745781, 42.87837875, 55.15927549, 64.82051021,
       71.00587821, 72.0165398 , 67.16011358, 59.38558999, 47.48847888,
       36.90779794, 41.20515048, 50.75214426, 61.67037472, 65.31650699,
       68.88477844, 60.54600942, 57.54590646, 44.76406371, 35.39105198,
       40.6445502 , 41.33378203, 53.517802  , 46.83745805, 50.72552797,
       38.75997496, 36.45649958, 40.77835526, 38.92980297, 43.12143542,
       37.34163296, 37.44374091, 47.91184761, 50.78566104, 64.14603784,
       58.8264265 , 60.99872726, 46.7319472 , 42.32869596, 43.30802753,
       52.99146035, 66.50374599, 71.31416061, 73.09228263, 67.49970524,
       59.92346891, 45.51629814, 37.34562691, 41.87023972, 47.3514015 ,
       63.31708183, 65.36714899, 69.51534388, 62.13701093, 56.93469625,
       41.83106415, 36.23612197, 41.43763218, 41.48953594, 51.21

In [41]:
import numpy.linalg as la

In [42]:
la.norm(curr_um.getDesignParameters() - rest_height_optimizer.applyTransformation(rest_height_optimizer.params()))

0.0